In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# Strategy and Account Parameters
initial_capital = 1_000_000
win_rate = 0.38
risk_reward_ratio = 3  # 1:3
risk_per_trade = 2000
reward_per_trade = risk_per_trade * risk_reward_ratio
trade_allocation = 10_000
max_concurrent_trades = 100
total_trades_to_simulate = 1200
simulation_runs = 1000

# Function to simulate a single Monte Carlo run
def simulate_trades():
    capital = initial_capital
    equity_curve = [capital]
    active_trades = []
    trade_results = []
    trade_timings = []

    current_day = 0
    while len(trade_results) < total_trades_to_simulate and current_day<365:
        # Close finished trades
        active_trades = [(p, d) for p, d in active_trades if d > current_day]

        # Slots available
        slots_available = max_concurrent_trades - len(active_trades)

        if slots_available > 0:
            trades_to_open = min(slots_available, total_trades_to_simulate - len(trade_results))

            for _ in range(trades_to_open):
                result = 1 if np.random.rand() < win_rate else -1
                profit_or_loss = reward_per_trade if result == 1 else -risk_per_trade
                duration = np.random.randint(30, 61)  # Trade lasts 30-60 days

                trade_results.append(profit_or_loss)
                active_trades.append((profit_or_loss, current_day + duration))
                trade_timings.append(current_day + duration)

        current_day += 1

    # Build equity curve over time
    capital = initial_capital
    trades_by_day = {}
    for p, d in zip(trade_results, trade_timings):
        trades_by_day.setdefault(d, []).append(p)
    return trade_results

# Run Simulations and Collect Results
all_results = []

for _ in range(simulation_runs):
    results=simulate_trades()
    all_results.append(results)

# Write Results to CSV
output_file = 'simulation_results.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(all_results)

print(f"Simulation results have been saved to {output_file}")